# Obtencion de propiedades de [atbuenosaires.com](https://www.atbuenosaires.com/Alq-Temporario)

*nota: La pagina tiene bastante codigo de JQuery y ajax*

Lo primero que hice fue explorar la manera en la que carga las propiedades cuando realiza una busqueda.

En la pagina principal tienen un buscador en el que se puede seleccionar el tipo de operacion(venta, alquiler, alq temporario), el tipo de propiedad (depto, casa ...), y la locacion/ubicacion.

Este ultimo parametro tiene algo particular porque solo encuentra cosas cuando se selecciona algo de las sugerencias, pero no rompe cuando se pone un texto cualquiera.

La busqueda llama la funcion `do_search()`  la cual construye un link de manera dinamica concatenando texto de los parametros del form. 
Esto es bastante feo.

Por otro lado cuando se va a la pestania de alquiler temporario la pagina va cargando de a poco las propiedades a medida que se scrollea hacia abajo.
Esto mediante un llamado de la funcion `get_next_page` la cual pide un query de ajax incrementando el numero de pagina y recibe una lista de objetos `<il>`
con la informacion de la propiedad. 
Esto resultados los agrega al nodo de  `id=propiedades` y cuando el numero de pagina es muy grande el query devuelve una pagina con  `<body>--NoMoreProperties-- </body>` 

Un nodo de la lista tiene la siguiente estructura
```html
<li>
    <a href="/p/378717-Departamento-en-Alquiler-temporario-en-San-Telmo-Perú-al-1300">
        <div class="prop-img">

            <div class="detalleColor resultadosB-favoritaSi animated bounceIn star-378717"><i class="fa fa-star"></i>
            </div>
            <div class="resultadosB-favoritaNo animated fadeInUp"><i class="fa fa-star-o"></i></div>
            <div class="prop-data">
                <div>35 m²</div><img
                    src="https://static.tokkobroker.com/tfw/img/prop-icons/suptotalconst.png?20190705061504">
            </div>
            <div class="prop-data2">
                <div>1</div><img src="https://static.tokkobroker.com/tfw/img/prop-icons/ambientes.png?20190705061504">
            </div>
            <div class="prop-img-bg animated fadeIn ">
                <div class="ver-ficha-box animated fadeInUp">VER DETALLES</div>
            </div><img class="dest-img"
                src="https://static.tokkobroker.com/water_pics/41766741736377234998772001103543741532070269742642577056108297622404199955171.jpg"
                alt="Foto Departamento en Alquiler temporario en  San Telmo ,  Capital Federal  Perú al 1300"
                title="Foto Departamento en Alquiler temporario en  San Telmo ,  Capital Federal  Perú al 1300">
        </div>

        <div class="prop-desc">
            <div class="prop-desc-tipo-ub">Departamento en Alquiler temporario en San Telmo , Capital Federal </div>
            <div class="prop-desc-dir">Perú al 1300</div>
        </div>
    </a>
    <div class="prop-valor-nro" style="cursor:pointer" onclick="star_unstar_prop(378717)">$19.500<div
            class="codref detalleColorText">AOT1410 AP378717</div>
        <div class="detalleColor resultadosB-interesa animated fadeInUp"><i class="fa fa-star"></i> +/- Favorito</div>
    </div>
</li>
```

A partir de estos queries puedo obtener todas las propiedades en alquiler temporario y a partir de estas los links a cada pagina con la informacion completa.

Por inspeccion manual la pagina 0 no trae nada, la pagina 1 trae una pagina completa para renderizar y a partir de la pagina 2 trae una pagina con unicamente los elementos de la lista para agregar.

In [0]:
import requests
from bs4 import BeautifulSoup

HOME_PATH = 'https://www.atbuenosaires.com'
query = HOME_PATH + '/Alq-Temporario?q=&currency=ANY&min-price=&max-price=&min-roofed=&max-roofed=&min-surface=&max-surface=&min-total_surface=&max-total_surface=&age=&min-age=&max-age=&suites=&rooms=&tags=&operation=&locations=&location_type=&ptypes=&o=2,2&watermark=&p={page}'

In [2]:
nodes = []

response = requests.get(query.format(page=1))
bs = BeautifulSoup(response.content)
nodes.extend( bs.find(id='propiedades').find_all('li') )

page = 2
while True:
  response = requests.get(query.format(page=page))
  bs = BeautifulSoup(response.content)
  elements = bs.find_all('li') 
  if elements:
    nodes.extend( elements )
    print('page:', page, 'with ', len(elements),'nodes')
    page += 1
  else:
    
    print('break at page: ',page)
    break

page: 2 with  20 nodes
page: 3 with  20 nodes
page: 4 with  20 nodes
page: 5 with  20 nodes
page: 6 with  20 nodes
page: 7 with  20 nodes
page: 8 with  20 nodes
page: 9 with  20 nodes
page: 10 with  20 nodes
page: 11 with  20 nodes
page: 12 with  20 nodes
page: 13 with  20 nodes
page: 14 with  20 nodes
page: 15 with  20 nodes
page: 16 with  20 nodes
page: 17 with  20 nodes
page: 18 with  20 nodes
page: 19 with  20 nodes
page: 20 with  20 nodes
page: 21 with  20 nodes
page: 22 with  20 nodes
page: 23 with  20 nodes
page: 24 with  20 nodes
page: 25 with  20 nodes
page: 26 with  20 nodes
page: 27 with  20 nodes
page: 28 with  20 nodes
page: 29 with  20 nodes
page: 30 with  20 nodes
page: 31 with  20 nodes
page: 32 with  20 nodes
page: 33 with  20 nodes
page: 34 with  20 nodes
page: 35 with  20 nodes
page: 36 with  20 nodes
page: 37 with  20 nodes
page: 38 with  20 nodes
page: 39 with  20 nodes
page: 40 with  20 nodes
page: 41 with  20 nodes
page: 42 with  20 nodes
page: 43 with  20 nodes


In [3]:
len(nodes)

1113

In [0]:
# para obtener el link del departamento
import urllib
n = nodes[0]
def get_link(node):
  link_propiedad = HOME_PATH + node.a['href']
  return link_propiedad

def get_operation_val(bs):
  return [t.text for t in bs.find(class_='operation-val').find_all('span') ]

def monto(bs):  
  monto, periodo = get_operation_val(bs)
  return {'monto':monto, 'periodo' : periodo}

def geo(bs):
    link = bs.find(id='properati_iframe_point').attrs['src']
    _,query = urllib.parse.splitquery(link)
    return dict( tuple(t.split('=')) for t in query.split('&') )

In [0]:
nombre_fichas = ['ficha_detalle_mobile',
                 'ficha_desc',
                 'ficha_extra',
                 'ficha_servicios',
                 'ficha_ambientes',
                 'ficha_adicionales'
                ]

def fichas(bs):  
  fichas = {}
  for nombre in nombre_fichas:
    ficha = bs.find(id=nombre)
    if ficha:
      fichas[nombre] = ficha
  return fichas

def get_bs(link_propiedad):
  response = requests.get(link_propiedad)
  bs = BeautifulSoup(response.content)
  return bs

def get_propiedad(node):
  link_propiedad = get_link(node)
  bs = get_bs(link_propiedad)
  return {link_propiedad: bs } 

In [6]:
propiedades = {} 

for i,node in enumerate(nodes):
  propiedades.update( get_propiedad(node) ) 
  n = len(nodes)-i
  print('\r',n,' ',n*'#',sep='',end='')

1 #

In [0]:
info = {}
errs = {}
for link_propiedad, bs in propiedades.items():
  fs = fichas(bs)
  fs.update( geo(bs) )
  try:
    fs.update( monto(bs) )
  except ValueError:
    errs.update({link_propiedad:get_operation_val(bs)})
    

  info.update( {link_propiedad:fs})


In [0]:
ficha = info["https://www.atbuenosaires.com/p/242779-Departamento-en-Alquiler-temporario-en-Belgrano-O'Higgins-al-2500,-esquina-Monroe."]

In [9]:
from IPython.display import HTML
HTML(str(ficha['ficha_desc']))

In [0]:
import bs4
def attribute(string):
  return dict( [ tuple( t.strip() for t in string.split(':') ) ] )

def extract_attributes(nodes):
  attributes = []
  for item in nodes:
      info = {}
      tick, string = item.contents
      info.update( tick.attrs )
      info.update( attribute(string) )
      attributes.append(info)
  return attributes

def basic_description(ficha):
  nodes = ficha['ficha_desc'].find(id='lista_informacion_basica').find_all('li')
  return extract_attributes(nodes)

def superficie(ficha):
  nodes = ficha['ficha_desc'].find(id='lista_superficies').find_all('li')
  return extract_attributes(nodes)
  
def detalle(ficha):
  contents = [ t.contents 
               for t in 
               ficha['ficha_detalle_mobile'].find_all(class_='ficha_detalle_item')]
  attributes = [{k.string:v} for k,_,v in contents]
  return attributes

def descripcion(ficha):
  return {'Descripcion':[ t for t in ficha['ficha_desc'].p.contents
           if isinstance(t,bs4.element.NavigableString)]} 

def informacion(ficha):
    return ( basic_description(ficha)
             + detalle(ficha)
             + superficie(ficha) 
             + [descripcion(ficha) ] )

In [11]:
import pandas as pd
pd.DataFrame( informacion(ficha) )

,Ambientes,Amoblado,Antigüedad,Barrio,Baños,Condición,Descripcion,Dirección,Disposición,Dormitorios,Expensas,Situación,Superficie Cubierta,Superficie Descubierta,Superficie Total,Total construido,class
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
2,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
3,NaN,NaN,NaN,NaN,NaN,Muy bueno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
4,NaN,NaN,5 Años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Habitada,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$1,NaN,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contrafrente,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"O'Higgins al 2500, esquina Monroe.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,Belgrano,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
HTML(str(ficha['ficha_detalle_mobile']))

In [13]:
HTML(str(ficha['ficha_desc']))

In [0]:
# HTML(str(ficha['ficha_extra']))

In [0]:
def extra(ficha):
  attributes = []
  for t in ficha['ficha_extra'].find_all('li'):
      k, v = t.contents[0].split(':') 
      k, v = k.strip(),v.strip()
      attributes.append({k:v})
  return attributes

# extra(ficha)

In [16]:
HTML(str(ficha['ficha_servicios']))

In [17]:
def extract_items(nodes):
  attributes = []
  for item in nodes:
      info = {}
      tick, string = item.contents
      info.update( tick.attrs )
      info.update( {string:'Si'} )
      attributes.append(info)
  return attributes

def servicios(ficha):
  nodes = ficha['ficha_servicios'].find_all('li')
  return extract_items(nodes)

servicios(ficha)

[{'Gas Natural': 'Si', 'class': ['fa', 'fa-check', 'detalleColorC']},
 {'Cable': 'Si', 'class': ['fa', 'fa-check', 'detalleColorC']},
 {'WiFi': 'Si', 'class': ['fa', 'fa-check', 'detalleColorC']}]

In [18]:
HTML(str(ficha['ficha_ambientes']))

In [19]:
def ambientes(ficha):
  nodes = ficha['ficha_ambientes'].find_all('li')
  return extract_items(nodes)

ambientes(ficha)

[{'Balcón': 'Si', 'class': ['fa', 'fa-check', 'detalleColorC']}]

In [20]:
HTML(str(ficha['ficha_adicionales']))

In [21]:
def adicionales(ficha):
  nodes = ficha['ficha_adicionales'].find_all('li')
  return extract_items(nodes)

adicionales(ficha)

[{'Aire Acondicionado individual': 'Si',
  'class': ['fa', 'fa-check', 'detalleColorC']},
 {'Amoblado': 'Si', 'class': ['fa', 'fa-check', 'detalleColorC']}]

In [22]:
def informacion_extra(ficha):
    info = []
    funcs = [extra, servicios, ambientes, adicionales]
    for f in funcs:
      try:
        info.extend( f(ficha) )
      except KeyError:
        print(f'No {f.__name__}')
    return info
  

pd.DataFrame( informacion_extra(ficha) )

No extra


,Aire Acondicionado individual,Amoblado,Balcón,Cable,Gas Natural,WiFi,class
0,NaN,NaN,NaN,NaN,Si,NaN,"[fa, fa-check, detalleColorC]"
1,NaN,NaN,NaN,Si,NaN,NaN,"[fa, fa-check, detalleColorC]"
2,NaN,NaN,NaN,NaN,NaN,Si,"[fa, fa-check, detalleColorC]"
3,NaN,NaN,Si,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
4,Si,NaN,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"
5,NaN,Si,NaN,NaN,NaN,NaN,"[fa, fa-check, detalleColorC]"


In [23]:
def record(ficha):
  record = {}
  for d in informacion_extra(ficha) + informacion(ficha):
    record.update(d)
  return record
record(ficha)

No extra


{'Aire Acondicionado individual': 'Si',
 'Ambientes': '1',
 'Amoblado': 'Si',
 'Antigüedad': '5 Años',
 'Balcón': 'Si',
 'Barrio': 'Belgrano',
 'Baños': '1',
 'Cable': 'Si',
 'Condición': 'Muy bueno',
 'Descripcion': ['1 ambiente. Amplio, luminoso, con balcón!',
  'Muy buen estado de conservación. Ideal 2 pax.',
  'Baño completo. Cocina equipada. Buen mobiliario!',
  "MUY BUENA UBICACIÓN!! A METROS DEL SUBTE ''D''.",
  'Ubicado en Belgrano. Hermoso barrio, de gran poder adquisitivo. Caracterizado por su gran oferta gastronomica y cultural. Importantes Avenidas de gran conectividad: Av. Cablido, Av. Monroe, Av del Libertador donde se encuentran innumerables medios de transporte (subtes, colectivos y trenes)',
  'VALOR PUBLICADO MAS SERVICIOS',
  'Condiciones para ingresar: ',
  '1 mes de adelanto - 1 mes de deposito - gastos adm.',
  'AOT1410 AP242779'],
 'Dirección': "O'Higgins al 2500, esquina Monroe.",
 'Disposición': 'Contrafrente',
 'Dormitorios': '1',
 'Expensas': '1',
 'Gas Natur

In [24]:
recs = []
for link, ficha in info.items():
    r = {'link':link}
    r.update(record(ficha))
    try:
      r['monto'] = ficha['monto']
      r['periodo'] = ficha['periodo']
      r['lat'] = ficha['lat']
      r['lon'] = ficha['lon']
    except KeyError as e:
      print(link,e)
    recs.append(r)

No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No ambientes
No extra
No extra
No extra
No extra
No extra
https://www.atbuenosaires.com/p/103794-Departamento-en-Alquiler-temporario-en-Centro-Av.-de-Mayo-al-1300 'monto'
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No servicios
No extra
No extra
No extra
No extra
No ambientes
No extra
No extra
No extra
No servicios
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No servicios
No extra
No extra
No extra
No ambientes
No extra
No extra
No extra
No extra
No extra
No extra
No extra
https://www.atbuenosaires.com/p/192909-Departamento-en-Alquiler-temporario-en-Versalles-Alvarez-Jonte-al-5600 'monto'
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extra
No extr

In [25]:
df[['Superficie Cubierta', 'Superficie Descubierta',
     'Superficie Semicubierta', 'Superficie Total']]

NameError: ignored

In [0]:
df = pd.DataFrame(recs)
df.lon = df.lon.astype(float)
df.lat = df.lat.astype(float)
df.monto = pd.to_numeric(df.monto.str.replace('$',''),errors='coerce')
df.Ambientes = df.Ambientes.astype(int)

##
df.Antigüedad = df.Antigüedad.str.replace('Años','').str.replace('Año','')
df.loc[df.Antigüedad == 'A Estrenar','Antigüedad'] = '0'
df.loc[df.Antigüedad == 'En Construcción','Antigüedad'] = '-1'
df.Antigüedad = df.Antigüedad.astype(int)
    
##
df['Superficie Total'] = df['Superficie Total'].str.replace('m²','').astype(float)
df['Superficie Descubierta'] = df['Superficie Descubierta'].str.replace('m²','').astype(float)
df['Superficie Semicubierta'] = df['Superficie Semicubierta'].str.replace('m²','').astype(float)
df['Superficie Cubierta'] = df['Superficie Cubierta'].str.replace('m²','').astype(float)

##
df.Parrilla = df.Parrilla.notna()

In [0]:
df.plot(x='lat',y='lon',kind='scatter')

In [0]:
import altair as alt
c = alt.Chart(df)

In [0]:
c.mark_circle().encode(
  latitude='lat:Q',
  longitude='lon:Q',
  size='Superficie Total:Q',
  color='monto:Q'
)

In [0]:
import seaborn as sns
sns.pairplot(df[['Antigüedad','monto','Ambientes','Superficie Total']])

In [0]:
sns.pairplot(df.loc[df.Parrilla.notna(),['Ambientes','monto','Superficie Cubierta', 'Superficie Descubierta',
     'Superficie Semicubierta', 'Superficie Total']])

In [0]:
c.mark_circle().encode(
    alt.X('monto:Q', bin=True),
    alt.Y('Superficie Total:Q', bin=True),
    size='count()',
    color='Ambientes:N'
)


In [0]:
c.mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Ambientes:N'
).properties(
    width=150,
    height=150
).repeat(
    row=['monto'],
    column=['monto', 'Superficie Cubierta', 'Superficie Descubierta',
     'Superficie Semicubierta', 'Superficie Total']
).interactive()

In [0]:
c.mark_area(
    opacity=0.3,
    interpolate='step'
).encode(
    alt.X('monto:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None),
    alt.Color('Ambientes:N')
).facet(
    column='Parrilla:N'
)
